In [1]:
import pandas as pd
import numpy as np
import preprocessing as prep
from ds_weather_api import *
import pickle
from datetime import datetime, timedelta, date

In [2]:
df = pd.read_csv("./Data/new_york/year_lga_dep.csv")
df.head()

,Unnamed: 0,Unnamed: 0.1,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,...,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,Unnamed: 52,DAY_OF_YEAR
0,425,48919,2018,2,5,1,2,2018-05-01,UA,N826UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1_5
1,483,450934,2018,2,5,1,2,2018-05-01,AA,N992NN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1_5
2,539,396682,2018,2,5,1,2,2018-05-01,AA,N813NN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1_5
3,599,358946,2018,2,5,1,2,2018-05-01,YX,N101HQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1_5
4,643,49039,2018,2,5,1,2,2018-05-01,UA,N446UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1_5


In [3]:
div_info_cols = [col for col in df.columns if col.startswith('Div') and col != 'Diverted']
df.drop(div_info_cols, axis = 1, inplace = True)
df.head()

,Unnamed: 0,Unnamed: 0.1,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,...,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,Unnamed: 52,DAY_OF_YEAR
0,425,48919,2018,2,5,1,2,2018-05-01,UA,N826UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1_5
1,483,450934,2018,2,5,1,2,2018-05-01,AA,N992NN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1_5
2,539,396682,2018,2,5,1,2,2018-05-01,AA,N813NN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1_5
3,599,358946,2018,2,5,1,2,2018-05-01,YX,N101HQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1_5
4,643,49039,2018,2,5,1,2,2018-05-01,UA,N446UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1_5


In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186782 entries, 0 to 186781
Data columns (total 56 columns):
Unnamed: 0               186782 non-null int64
Unnamed: 0.1             186782 non-null int64
YEAR                     186782 non-null int64
QUARTER                  186782 non-null int64
MONTH                    186782 non-null int64
DAY_OF_MONTH             186782 non-null int64
DAY_OF_WEEK              186782 non-null int64
FL_DATE                  186782 non-null object
OP_UNIQUE_CARRIER        186782 non-null object
TAIL_NUM                 186461 non-null object
OP_CARRIER_FL_NUM        186782 non-null int64
ORIGIN_CITY_MARKET_ID    186782 non-null int64
ORIGIN                   186782 non-null object
ORIGIN_CITY_NAME         186782 non-null object
ORIGIN_STATE_ABR         186782 non-null object
DEST_CITY_MARKET_ID      186782 non-null int64
DEST                     186782 non-null object
DEST_CITY_NAME           186782 non-null object
DEST_STATE_ABR           186782 non

In [5]:
df.describe()

,Unnamed: 0,Unnamed: 0.1,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_FL_NUM,ORIGIN_CITY_MARKET_ID,DEST_CITY_MARKET_ID,...,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,Unnamed: 52
count,1.867820e+05,186782.000000,186782.000000,186782.000000,186782.000000,186782.000000,186782.000000,186782.000000,186782.0,186782.000000,...,186782.000000,42279.000000,42279.000000,42279.000000,42279.000000,42279.000000,2039.000000,2039.000000,2039.000000,0.0
mean,3.957647e+06,316336.017261,2018.380626,2.465056,6.388442,15.794102,3.789637,3247.940423,31703.0,31666.062726,...,3.333742,15.619007,3.323328,19.235885,0.108375,34.391873,1406.237862,68.637567,66.730260,NaN
std,2.283181e+06,178548.121724,0.485542,1.070515,3.285386,8.732372,1.947076,1842.848356,0.0,1335.362490,...,1.565715,47.806668,30.144231,42.764522,6.493989,55.469840,448.791887,54.979063,52.009858,NaN
min,4.250000e+02,6.000000,2018.000000,1.000000,1.000000,1.000000,1.000000,16.000000,31703.0,30154.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,1.000000,1.000000,NaN
25%,1.998601e+06,160114.250000,2018.000000,2.000000,4.000000,8.000000,2.000000,1662.000000,31703.0,30721.000000,...,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1059.000000,24.000000,24.000000,NaN
50%,3.950142e+06,334438.500000,2018.000000,2.000000,6.000000,16.000000,4.000000,3440.000000,31703.0,31057.000000,...,3.000000,0.000000,0.000000,7.000000,0.000000,14.000000,1449.000000,50.000000,49.000000,NaN
75%,5.937538e+06,462576.750000,2019.000000,3.000000,9.000000,23.000000,5.000000,4762.000000,31703.0,32467.000000,...,5.000000,12.000000,0.000000,23.000000,0.000000,48.000000,1736.000000,111.000000,109.000000,NaN
max,7.912045e+06,645141.000000,2019.000000,4.000000,12.000000,31.000000,7.000000,7881.000000,31703.0,35412.000000,...,9.000000,1556.000000,1203.000000,1403.000000,927.000000,1411.000000,2342.000000,302.000000,258.000000,NaN


In [6]:
prep.print_df_metadata(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186782 entries, 0 to 186781
Data columns (total 57 columns):
Unnamed: 0               186782 non-null int64
Unnamed: 0.1             186782 non-null int64
YEAR                     186782 non-null int64
QUARTER                  186782 non-null int64
MONTH                    186782 non-null int64
DAY_OF_MONTH             186782 non-null int64
DAY_OF_WEEK              186782 non-null int64
FL_DATE                  186782 non-null object
OP_UNIQUE_CARRIER        186782 non-null object
TAIL_NUM                 186461 non-null object
OP_CARRIER_FL_NUM        186782 non-null int64
ORIGIN_CITY_MARKET_ID    186782 non-null int64
ORIGIN                   186782 non-null object
ORIGIN_CITY_NAME         186782 non-null object
ORIGIN_STATE_ABR         186782 non-null object
DEST_CITY_MARKET_ID      186782 non-null int64
DEST                     186782 non-null object
DEST_CITY_NAME           186782 non-null object
DEST_STATE_ABR           186782 non

Column: ACTUAL_ELAPSED_TIME  |  Type = float64  |  341 Unique Values  
	 Negative Count = 0  |  NaN count = 6239
Column: AIR_TIME  |  Type = float64  |  273 Unique Values  
	 Negative Count = 0  |  NaN count = 6239
Column: FLIGHTS  |  Type = float64  |  1 Unique Values  
	 Negative Count = 0  |  NaN count = 0
Column: DISTANCE  |  Type = float64  |  77 Unique Values  
	 Negative Count = 0  |  NaN count = 0
Column: DISTANCE_GROUP  |  Type = int64  |  9 Unique Values  
	 Negative Count = 0  |  NaN count = 0
Column: CARRIER_DELAY  |  Type = float64  |  430 Unique Values  
	 Negative Count = 0  |  NaN count = 144503
Column: WEATHER_DELAY  |  Type = float64  |  324 Unique Values  
	 Negative Count = 0  |  NaN count = 144503
Column: NAS_DELAY  |  Type = float64  |  394 Unique Values  
	 Negative Count = 0  |  NaN count = 144503
Column: SECURITY_DELAY  |  Type = float64  |  52 Unique Values  
	 Negative Count = 0  |  NaN count = 144503
Column: LATE_AIRCRAFT_DELAY  |  Type = float64  |  409 Uni

In [7]:
df.head()

,Unnamed: 0,Unnamed: 0.1,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,...,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,Unnamed: 52,DAY_OF_YEAR
0,425,48919,2018,2,5,1,2,2018-05-01,UA,N826UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1_5
1,483,450934,2018,2,5,1,2,2018-05-01,AA,N992NN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1_5
2,539,396682,2018,2,5,1,2,2018-05-01,AA,N813NN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1_5
3,599,358946,2018,2,5,1,2,2018-05-01,YX,N101HQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1_5
4,643,49039,2018,2,5,1,2,2018-05-01,UA,N446UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1_5


In [8]:
with open('Data/WeatherData_May2018-May2019.bin','rb') as f:
    weather_data = pickle.load(f)
weather_data

[{<WeatherReportType.Daily: 'daily'>: {datetime.datetime(2018, 4, 29, 4, 0): (time: 2018-04-29 04:00:00, summary: Drizzle in the morning., humidity: 0.68, pressure: 1013.07, visibility: 15.614)},
  <WeatherReportType.Hourly: 'hourly'>: {datetime.datetime(2018, 4, 29, 4, 0): (time: 2018-04-29 04:00:00, summary: Drizzle, humidity: 0.89, pressure: 1011.59, visibility: 15.203),
   datetime.datetime(2018, 4, 29, 5, 0): (time: 2018-04-29 05:00:00, summary: Drizzle, humidity: 0.9, pressure: 1011.43, visibility: 14.71),
   datetime.datetime(2018, 4, 29, 6, 0): (time: 2018-04-29 06:00:00, summary: Drizzle, humidity: 0.91, pressure: 1010.88, visibility: 13.947),
   datetime.datetime(2018, 4, 29, 7, 0): (time: 2018-04-29 07:00:00, summary: Drizzle, humidity: 0.92, pressure: 1011.16, visibility: 14.628),
   datetime.datetime(2018, 4, 29, 8, 0): (time: 2018-04-29 08:00:00, summary: Drizzle, humidity: 0.91, pressure: 1011.38, visibility: 12.855),
   datetime.datetime(2018, 4, 29, 9, 0): (time: 2018-

In [9]:
weather_data[0][WeatherReportType.Daily]

{datetime.datetime(2018, 4, 29, 4, 0): (time: 2018-04-29 04:00:00, summary: Drizzle in the morning., humidity: 0.68, pressure: 1013.07, visibility: 15.614)}

In [10]:
sorted(weather_data[0][WeatherReportType.Hourly].keys())

[datetime.datetime(2018, 4, 29, 4, 0),
 datetime.datetime(2018, 4, 29, 5, 0),
 datetime.datetime(2018, 4, 29, 6, 0),
 datetime.datetime(2018, 4, 29, 7, 0),
 datetime.datetime(2018, 4, 29, 8, 0),
 datetime.datetime(2018, 4, 29, 9, 0),
 datetime.datetime(2018, 4, 29, 10, 0),
 datetime.datetime(2018, 4, 29, 11, 0),
 datetime.datetime(2018, 4, 29, 12, 0),
 datetime.datetime(2018, 4, 29, 13, 0),
 datetime.datetime(2018, 4, 29, 14, 0),
 datetime.datetime(2018, 4, 29, 15, 0),
 datetime.datetime(2018, 4, 29, 16, 0),
 datetime.datetime(2018, 4, 29, 17, 0),
 datetime.datetime(2018, 4, 29, 18, 0),
 datetime.datetime(2018, 4, 29, 19, 0),
 datetime.datetime(2018, 4, 29, 20, 0),
 datetime.datetime(2018, 4, 29, 21, 0),
 datetime.datetime(2018, 4, 29, 22, 0),
 datetime.datetime(2018, 4, 29, 23, 0),
 datetime.datetime(2018, 4, 30, 0, 0),
 datetime.datetime(2018, 4, 30, 1, 0),
 datetime.datetime(2018, 4, 30, 2, 0),
 datetime.datetime(2018, 4, 30, 3, 0)]

In [21]:
date_to_hours = {}
for data in weather_data:
    hourly_data = data[WeatherReportType.Hourly]
    
    for val in data[WeatherReportType.Daily].values():
        daily = val
        break
        
    hours = sorted(hourly_data.keys())
    for hour in hours:
        dt = hour.date()
        if dt not in date_to_hours:
            date_to_hours[dt] = {}
        date_to_hours[dt][hour.hour] = hourly_data[hour].extend_with(daily)

date_to_hours

{datetime.date(2018, 4, 29): {4: (time: 2018-04-29 04:00:00, summary: Drizzle, humidity: 0.89, pressure: 1011.59, visibility: 15.203),
  5: (time: 2018-04-29 05:00:00, summary: Drizzle, humidity: 0.9, pressure: 1011.43, visibility: 14.71),
  6: (time: 2018-04-29 06:00:00, summary: Drizzle, humidity: 0.91, pressure: 1010.88, visibility: 13.947),
  7: (time: 2018-04-29 07:00:00, summary: Drizzle, humidity: 0.92, pressure: 1011.16, visibility: 14.628),
  8: (time: 2018-04-29 08:00:00, summary: Drizzle, humidity: 0.91, pressure: 1011.38, visibility: 12.855),
  9: (time: 2018-04-29 09:00:00, summary: Drizzle, humidity: 0.91, pressure: 1010.84, visibility: 15.087),
  10: (time: 2018-04-29 10:00:00, summary: Possible Drizzle, humidity: 0.89, pressure: 1010.88, visibility: 15.485),
  11: (time: 2018-04-29 11:00:00, summary: Drizzle, humidity: 0.86, pressure: 1011.68, visibility: 15.403),
  12: (time: 2018-04-29 12:00:00, summary: Possible Drizzle, humidity: 0.81, pressure: 1011.92, visibility:

In [22]:
weather_data[0]

{<WeatherReportType.Daily: 'daily'>: {datetime.datetime(2018, 4, 29, 4, 0): (time: 2018-04-29 04:00:00, summary: Drizzle in the morning., humidity: 0.68, pressure: 1013.07, visibility: 15.614)},
 <WeatherReportType.Hourly: 'hourly'>: {datetime.datetime(2018, 4, 29, 4, 0): (time: 2018-04-29 04:00:00, summary: Drizzle, humidity: 0.89, pressure: 1011.59, visibility: 15.203),
  datetime.datetime(2018, 4, 29, 5, 0): (time: 2018-04-29 05:00:00, summary: Drizzle, humidity: 0.9, pressure: 1011.43, visibility: 14.71),
  datetime.datetime(2018, 4, 29, 6, 0): (time: 2018-04-29 06:00:00, summary: Drizzle, humidity: 0.91, pressure: 1010.88, visibility: 13.947),
  datetime.datetime(2018, 4, 29, 7, 0): (time: 2018-04-29 07:00:00, summary: Drizzle, humidity: 0.92, pressure: 1011.16, visibility: 14.628),
  datetime.datetime(2018, 4, 29, 8, 0): (time: 2018-04-29 08:00:00, summary: Drizzle, humidity: 0.91, pressure: 1011.38, visibility: 12.855),
  datetime.datetime(2018, 4, 29, 9, 0): (time: 2018-04-29 0

In [25]:
def get_weather_attribute_function(attr_name):
    def get_attribute_by_time(df):
        dt = date.fromisoformat(df['FL_DATE'])
        hour = int(np.round(df['CRS_DEP_TIME'] / 100.))
        
        if dt not in date_to_hours or hour not in date_to_hours[dt]:
            return np.nan
        return date_to_hours[dt][hour].__getattribute__(attr_name)
    return get_attribute_by_time

for attr_name in ['weather_label', 'precipitation_intensity','precipitation_probability', 'visibility', 
                  'cloud_cover', 'humidity', 'wind_bearing', 'wind_speed', 'uv_index']:
    df[attr_name] = df.apply(get_weather_attribute_function(attr_name), axis = 1)

,Unnamed: 0,Unnamed: 0.1,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,...,DAY_OF_YEAR,weather_label,precipitation_intensity,precipitation_probability,visibility,cloud_cover,humidity,wind_bearing,wind_speed,uv_index
0,425,48919,2018,2,5,1,2,2018-05-01,UA,N826UA,...,1_5,WeatherLabel.PartlyCloudyNight,NaN,NaN,16.089,NaN,0.77,NaN,NaN,NaN
1,483,450934,2018,2,5,1,2,2018-05-01,AA,N992NN,...,1_5,WeatherLabel.PartlyCloudyNight,NaN,NaN,16.089,NaN,0.77,NaN,NaN,NaN
2,539,396682,2018,2,5,1,2,2018-05-01,AA,N813NN,...,1_5,WeatherLabel.PartlyCloudyNight,NaN,NaN,16.089,NaN,0.78,NaN,NaN,NaN
3,599,358946,2018,2,5,1,2,2018-05-01,YX,N101HQ,...,1_5,WeatherLabel.PartlyCloudyNight,NaN,NaN,16.089,NaN,0.78,NaN,NaN,NaN
4,643,49039,2018,2,5,1,2,2018-05-01,UA,N446UA,...,1_5,WeatherLabel.PartlyCloudyNight,NaN,NaN,16.089,NaN,0.78,NaN,NaN,NaN
5,786,226103,2018,2,5,1,2,2018-05-01,WN,N761RR,...,1_5,WeatherLabel.PartlyCloudyNight,NaN,NaN,16.089,NaN,0.78,NaN,NaN,NaN
6,787,226108,2018,2,5,1,2,2018-05-01,WN,N720WN,...,1_5,WeatherLabel.PartlyCloudyNight,NaN,NaN,16.089,NaN,0.78,NaN,NaN,NaN
7,825,359236,2018,2,5,1,2,2018-05-01,YX,N109HQ,...,1_5,WeatherLabel.PartlyCloudyNight,NaN,NaN,16.089,NaN,0.78,NaN,NaN,NaN
8,830,371446,2018,2,5,1,2,2018-05-01,YX,N204JQ,...,1_5,WeatherLabel.PartlyCloudyNight,NaN,NaN,16.089,NaN,0.78,NaN,NaN,NaN
9,833,371523,2018,2,5,1,2,2018-05-01,YX,N207JQ,...,1_5,WeatherLabel.PartlyCloudyNight,NaN,NaN,16.089,NaN,0.78,NaN,NaN,NaN


In [33]:
df.isnull().sum()

Unnamed: 0           0
Unnamed: 0.1         0
YEAR                 0
QUARTER              0
MONTH                0
                 ...  
cloud_cover     186782
humidity           263
wind_bearing    186782
wind_speed      186782
uv_index        186782
Length: 65, dtype: int64

In [63]:
import numpy as np
